<a href="https://colab.research.google.com/github/jacquelinevanarkel/CulturalEvolutionCollateralSignals-2021/blob/main/Model_Winter_%26_Wedel_(2016).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>